In [1]:
import copy

In [92]:
def e_and_bit(a, b):
    return (a & b)

def e_or_bit(a, b):
    return (a | b)

def e_xor_bit(a, b):
    return a ^ b

def e_nand_bit(a, b):
    return (1- (a & b))

def e_nor_bit(a, b):
    return (1- (a | b))

def e_xnor_bit(a, b):
    return (1- (a ^ b))

def e_not_bit(a):
    return (1- a)  # Ignores b

def e_mux_bit(sel, a, b):
    return (a & sel) | (b & (~sel & 1))

def e_and(a, b, result):
    result[:] = [x & y for x, y in zip(a, b)]

def e_or(a, b, result):
    result[:] = [x | y for x, y in zip(a, b)]

def e_xor(a, b, result):
    result[:] = [x ^ y for x, y in zip(a, b)]

def e_nand(a, b, result):
    result[:] = [~(x & y) & 1 for x, y in zip(a, b)]

def e_nor(a, b, result):
    result[:] = [~(x | y) & 1 for x, y in zip(a, b)]

def e_xnor(a, b, result):
    result[:] = [~(x ^ y) & 1 for x, y in zip(a, b)]

def e_not(a, result):
    result[:] = [~x & 1 for x in a]  # Ignores b

def e_mux(sel, a, b, result):
    result[:] = [(a[i] & (sel)) | (b[i] & (~sel & 1)) for i in range(len(a))]

def e_and_range(a, b, result, low, high):
    result[low:high] = [x & y for x, y in zip(a[low:high], b[low:high])]

def e_or_range(a, b, result, low, high):
    result[low:high] = [x | y for x, y in zip(a[low:high], b[low:high])]

def e_xor_range(a, b, result, low, high):
    result[low:high] = [x ^ y for x, y in zip(a[low:high], b[low:high])]

def e_nand_range(a, b, result, low, high):
    result[low:high] = [~(x & y) & 1 for x, y in zip(a[low:high], b[low:high])]

def e_nor_range(a, b, result, low, high):
    result[low:high] = [~(x | y) & 1 for x, y in zip(a[low:high], b[low:high])]

def e_xnor_range(a, b, result, low, high):
    result[low:high] = [~(x ^ y) & 1 for x, y in zip(a[low:high], b[low:high])]

def e_not_range(a, result, low, high):
    result[low:high] = [~x & 1 for x in a[low:high]]  # Ignores b

def e_shl_p(a, LSB):
    result = [LSB]
    for i in range(1, len(a)):
        result.append(a[i-1])
    return result


In [68]:
def int_to_bits(signed_int, bit_length=32):
    """
    Convert a signed integer to an array of bits with LSB (sign bit) at index 0.
    Uses two's complement representation.
    """
    # Handle negative numbers using two's complement
    if signed_int < 0:
        signed_int = (1 << bit_length) + signed_int
    # Convert to binary and remove '0b' prefix
    binary = bin(signed_int)[2:].zfill(bit_length)
    # Return bits as list with LSB (sign bit) at index 0
    return [int(bit) for bit in binary[::-1]]

def bits_to_int(bits):
    """
    Convert an array of bits with LSB (sign bit) at index 0 back to a signed integer.
    Assumes two's complement representation.
    """
    # Reverse bits to have MSB at highest index
    binary = ''.join(str(bit) for bit in bits[::-1])
    # Convert binary string to integer
    result = int(binary, 2)
    # Handle negative numbers (if sign bit is 1)
    bit_length = len(bits)
    if bits[bit_length-1] == 1:  # Sign bit is 1
        result -= (1 << bit_length)
    return result

In [69]:
def add_supplement(a, b, size):
    """
    Adds two bit lists a and b (using only the first 'size' bits of a if longer),
    returns the result list of length 'size' (mod 2^size, discarding overflow).
    """
    if size == 0:
        return []
    carry = [0] * (size + 1)
    temp = [0] * size
    for i in range(size):
        temp[i] = a[i] ^ b[i]
    for i in range(size):
        gen = a[i] & b[i]
        carry[i + 1] = gen | (temp[i] & carry[i])
    result = [0] * size
    for i in range(size):
        result[i] = temp[i] ^ carry[i]
    return result

In [70]:

def multiplier(a, b, result):
    """
    Computes the bit-wise multiplication of a and b (lists of bits, LSB at index 0),
    stores the low len(a) bits of the product in result (handles both unsigned and signed via modular arithmetic).
    Assumes len(a) == len(b) == len(result).
    """
    size = len(a)
    tmp_array = [0] * size
    sum_ = [0] * size
    temp_sum = [0] * size
    for i in range(size):
        # Reset tmp_array low bits (higher bits unused)
        for j in range(size - i):
            tmp_array[j] = a[i] & b[j]
        temp_sum[i:] = add_supplement(tmp_array, sum_[i:], size - i)
        sum_[:] = temp_sum[:]
    result[:] = sum_[:]

In [71]:
a = int_to_bits(10,  bit_length=8)
b = int_to_bits(-10, bit_length=8)
print(a[::-1])
print(b[::-1])
res = [0] * 8
multiplier(a, b, res)
print(res[::-1])
print(bits_to_int(res))

[0, 0, 0, 0, 1, 0, 1, 0]
[1, 1, 1, 1, 0, 1, 1, 0]
[1, 0, 0, 1, 1, 1, 0, 0]
-100


In [72]:
# -20 -41
# -14 -17
# -44 -29

In [73]:
def adder_c(a, b, result):
    size = len(a)
    carry = [0] * (size + 1)
    temp = [0] * size

    e_xor(a, b, temp)
    for i in range(size):
        result[i] = e_xor_bit(carry[i], temp[i])
        if i != size - 1:
            carry[i+1] = e_mux_bit(temp[i], carry[i], a[i])

In [74]:
def adder_rs(a, b, result):
    size = len(a)
    carry = [0] * (size + 1)
    temp = [0] * size
    a_and_b = [0] * size
    
    e_xor(a, b, temp)
    e_and(a, b, a_and_b)
    
    for i in range(size):
        result[i] = e_xor_bit(carry[i], temp[i])
        
        if i != size - 1:
            a_and_carry = e_and_bit(a[i], b[i])
            b_and_carry = e_and_bit(b[i], carry[i])
            temp_carry = e_or_bit(a_and_b[i], a_and_carry)
            carry[i+1] = e_or_bit(temp_carry, b_and_carry)
            

In [75]:
a = int_to_bits(-44,  bit_length=8)
b = int_to_bits(-29, bit_length=8)
print(a[::-1])
print(b[::-1])
res = [0] * 8
adder_c(a, b, res)
print(res[::-1])
print(bits_to_int(res))

[1, 1, 0, 1, 0, 1, 0, 0]
[1, 1, 1, 0, 0, 0, 1, 1]
[1, 0, 1, 1, 0, 1, 1, 1]
-73


In [76]:
a = int_to_bits(44,  bit_length=8)
b = int_to_bits(-29, bit_length=8)
print(a[::-1])
print(b[::-1])
res = [0] * 8
adder_rs(a, b, res)
print(res[::-1])
print(bits_to_int(res))

[0, 0, 1, 0, 1, 1, 0, 0]
[1, 1, 1, 0, 0, 0, 1, 1]
[0, 0, 0, 0, 1, 1, 1, 1]
15


In [77]:
def subtractor(a, b, result):
    size = len(a)
    borrow = [0] * size
    temp_0 = [0] * size
    temp_1 = [0] * size
    temp_2 = [0] * size
    
    result[0] = e_xor_bit(a[0], b[0])
    temp_0[0] = e_not_bit(a[0])
    borrow[0] = e_and_bit(temp_0[0], b[0])
    
    e_xor_range(a, b, temp_0, 1, size)
    e_not_range(a, temp_1, 1, size)
    
    for i in range(1, size):
        result[i] = e_xor_bit(temp_0[i], borrow[i-1])
        
        if i != size - 1:
            temp_2[i] = e_and_bit(temp_1[i], b[i])
            temp_0[i] = e_not_bit(temp_0[i])
            temp_1[i] = e_and_bit(borrow[i-1], temp_0[i])
            borrow[i] = e_or_bit(temp_2[i], temp_1[i])

In [78]:
a = int_to_bits(44,  bit_length=8)
b = int_to_bits(-29, bit_length=8)
print(a[::-1])
print(b[::-1])
res = [0] * 8
subtractor(a, b, res)
print(res[::-1])
print(bits_to_int(res))

[0, 0, 1, 0, 1, 1, 0, 0]
[1, 1, 1, 0, 0, 0, 1, 1]
[0, 1, 0, 0, 1, 0, 0, 1]
73


In [95]:
def divider(a, b, result):
    size = len(a)
    Q = a
    A = [0] * size
    M = b
    MSB = 0
    zero = 0
    A_msb = 0
    not_A_msb = 0
    A_tmp = [0] * size
    Q_tmp = [0] * size
    A_m = [0] * size
    
    for _ in range(size):
        Q_tmp = e_shl_p(Q, zero)
        MSB = Q[size-1]
        Q = Q_tmp
        print("\t\tQ1:", Q)
        A_tmp = e_shl_p(A, MSB)
        subtractor(A_tmp, M, A)
        A_msb = A[size-1]
        not_A_msb = e_not_bit(A_msb)
        Q[0] = not_A_msb
        print("\t\tQ2:", Q)
        adder_rs(A, M, A_m)
        e_mux(A_msb, A_m, A, A_tmp)
        A = A_tmp
        
    for i in range(size):
        result[i] = Q[i]

In [96]:
a = int_to_bits(8,  bit_length=8)
b = int_to_bits(2, bit_length=8)
print(a[::-1])
print(b[::-1])
res = [0] * 8
divider(a, b, res)
print(res[::-1])
print(bits_to_int(res))

[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
		Q1: [0, 0, 0, 1, 0, 0, 0, 0]
		Q2: [0, 0, 0, 1, 0, 0, 0, 0]
		Q1: [0, 0, 1, 0, 0, 0, 0, 0]
		Q2: [0, 0, 1, 0, 0, 0, 0, 0]
		Q1: [0, 1, 0, 0, 0, 0, 0, 0]
		Q2: [0, 1, 0, 0, 0, 0, 0, 0]
		Q1: [1, 0, 0, 0, 0, 0, 0, 0]
		Q2: [1, 0, 0, 0, 0, 0, 0, 0]
		Q1: [0, 0, 0, 0, 0, 0, 0, 0]
		Q2: [0, 0, 0, 0, 0, 0, 0, 1]
		Q1: [0, 0, 0, 0, 0, 0, 1, 0]
		Q2: [0, 0, 0, 0, 0, 0, 1, 1]
		Q1: [0, 0, 0, 0, 0, 1, 1, 0]
		Q2: [0, 0, 0, 0, 0, 1, 1, 1]
		Q1: [0, 0, 0, 0, 1, 1, 1, 0]
		Q2: [0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 1, 1, 1, 1]
15
